In [5]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

#time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190108'
#xml_dir = '/data/darknet/python/' + time_str + '_xml/'
xml_dir = '/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/'
SEED_XML_DIR = xml_dir
GENE_XML_DIR = SEED_XML_DIR


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']

for xml_name in seed_xml_names:    
    if(not xml_name.endswith('.xml')):
        continue
    if(not (int(xml_name[-15:-4]) >= 442000)):
        continue    
    #img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    print(SEED_XML_DIR + xml_name)
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)

    root = tree.getroot()
    size = root.find('size')   
    boxes = []
    b = []  
    #wrong_detection_objs = [[393, 408, 52, 83]]
    wrong_detection_objs = [[200, 233, 15, 48]]

    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.4):
                print(IOU(erro_bbox, b))
                print('removed')
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        '''    
    tree.write(GENE_XML_DIR + xml_name)

/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.316444_00000442005.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.331073_00000442006.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.529090_00000442019.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.557605_00000442021.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.572009_00000442022.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.600447_00000442024.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.614628_00000442025.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.628914_00000442026.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.643470_00000442027.xml
/data/dark

/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154134.840046_00000442750.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154134.937150_00000442756.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154134.984513_00000442759.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.111650_00000442767.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.191917_00000442772.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.240172_00000442775.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.256506_00000442776.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.289900_00000442778.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154135.306494_00000442779.xml
/data/dark

/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.500025_00000443963.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.549514_00000443966.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.613554_00000443970.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.649118_00000443972.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.665208_00000443973.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.746707_00000443976.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.780912_00000443978.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.796762_00000443979.xml
/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.813086_00000443980.xml
/data/dark